# EDA of Adult Data

Joel, Yaniv, Jeff<br>
Merck Challenge - Project 2<br>
Rutgers MSDS<br>
2/4/21<br>

Items to Check for EDA:  
1. Data loading and checking  
2a. Numerical summaries  
2b. Graphical summaries  
3. Extract important variable(s) and analyze them  
3a. Check for outliers and missing data  
(Note: Did not include a on outliers (yet), as should determine the threshold of removing outliers first. I would suggest checking the jackknife residuals (refer to HW #6 from reg & TS))(EDIT: might be different for GLMs...)  
3b. Develop and test simple models  
3c. Test model assumptions

Q: How much EDA should we do? (e.g. doing summaries of a few vs all variables, using some vs all of Yaniv's EDA code, etc.)

In [2]:
### Preliminaries
import pandas as pd # essential for data analysis
import numpy as np # gives tools for working with arrays
import matplotlib.pyplot as plt # useful for plots in general
import seaborn as sns # needed for histograms
import pylab as pyl #for generating plots: show()
import statsmodels.api as sm  #to use the Logit()
!pip install datawig
import datawig
import collections

In [9]:
# Loading and Checking Data
!pwd
  # type and enter pwd and see if dataset is in same directory
data = pd.read_csv("/Users/jeffgong/myProject/AI/MerckChallengeIMLRepo/adult.data")  # type and enter pwd and see if dataset is in same directory

#renaming columns
new_names =["age","workclass","fnlwgt","education","education.num","marital.status","occupation","relationship","race","sex","capital.gain","capital.loss","hours.per.week","native.country","salary"]


data = data.rename(columns={i:j for i,j in zip(data.columns,new_names)})


data[20:30]


/Users/jeffgong/myProject/AI/MerckChallengeIMLRepo


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,salary
20,54,Private,302146,HS-grad,9,Separated,Other-service,Unmarried,Black,Female,0,0,20,United-States,<=50K
21,35,Federal-gov,76845,9th,5,Married-civ-spouse,Farming-fishing,Husband,Black,Male,0,0,40,United-States,<=50K
22,43,Private,117037,11th,7,Married-civ-spouse,Transport-moving,Husband,White,Male,0,2042,40,United-States,<=50K
23,59,Private,109015,HS-grad,9,Divorced,Tech-support,Unmarried,White,Female,0,0,40,United-States,<=50K
24,56,Local-gov,216851,Bachelors,13,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,40,United-States,>50K
25,19,Private,168294,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K
26,54,?,180211,Some-college,10,Married-civ-spouse,?,Husband,Asian-Pac-Islander,Male,0,0,60,South,>50K
27,39,Private,367260,HS-grad,9,Divorced,Exec-managerial,Not-in-family,White,Male,0,0,80,United-States,<=50K
28,49,Private,193366,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
29,23,Local-gov,190709,Assoc-acdm,12,Never-married,Protective-serv,Not-in-family,White,Male,0,0,52,United-States,<=50K


In [10]:
data.describe()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,32560.000000,3.256000e+04,32560.000000,32560.000000,32560.000000,32560.000000
mean,38.581634,1.897818e+05,10.080590,1077.615172,87.306511,40.437469
std,13.640642,1.055498e+05,2.572709,7385.402999,402.966116,12.347618
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178315e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783630e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370545e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [11]:
data.shape # 299 rows, 13 cols

(32560, 15)

In [12]:
data.std()

age                   13.640642
fnlwgt            105549.764924
education.num          2.572709
capital.gain        7385.402999
capital.loss         402.966116
hours.per.week        12.347618
dtype: float64

In [13]:
data.info() # info about each variable


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32560 entries, 0 to 32559
Data columns (total 15 columns):
age               32560 non-null int64
workclass         32560 non-null object
fnlwgt            32560 non-null int64
education         32560 non-null object
education.num     32560 non-null int64
marital.status    32560 non-null object
occupation        32560 non-null object
relationship      32560 non-null object
race              32560 non-null object
sex               32560 non-null object
capital.gain      32560 non-null int64
capital.loss      32560 non-null int64
hours.per.week    32560 non-null int64
native.country    32560 non-null object
salary            32560 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [14]:
data.isnull().any() # check if any missing data
# replace ? values with NaN
#print(data.loc[[26]]) getting data from row 26


val = data.at[26,'workclass']
for col in data:
    data[col]= data[col].replace([val],np.nan)
print(data.isnull().any()) 
data[20:40]



age               False
workclass          True
fnlwgt            False
education         False
education.num     False
marital.status    False
occupation         True
relationship      False
race              False
sex               False
capital.gain      False
capital.loss      False
hours.per.week    False
native.country     True
salary            False
dtype: bool


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,salary
20,54,Private,302146,HS-grad,9,Separated,Other-service,Unmarried,Black,Female,0,0,20,United-States,<=50K
21,35,Federal-gov,76845,9th,5,Married-civ-spouse,Farming-fishing,Husband,Black,Male,0,0,40,United-States,<=50K
22,43,Private,117037,11th,7,Married-civ-spouse,Transport-moving,Husband,White,Male,0,2042,40,United-States,<=50K
23,59,Private,109015,HS-grad,9,Divorced,Tech-support,Unmarried,White,Female,0,0,40,United-States,<=50K
24,56,Local-gov,216851,Bachelors,13,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,40,United-States,>50K
25,19,Private,168294,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K
26,54,NaN,180211,Some-college,10,Married-civ-spouse,NaN,Husband,Asian-Pac-Islander,Male,0,0,60,South,>50K
27,39,Private,367260,HS-grad,9,Divorced,Exec-managerial,Not-in-family,White,Male,0,0,80,United-States,<=50K
28,49,Private,193366,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
29,23,Local-gov,190709,Assoc-acdm,12,Never-married,Protective-serv,Not-in-family,White,Male,0,0,52,United-States,<=50K


In [20]:
#skip this for now
#split dataframe w/ isna boolean operator on column workclass for imputation

#testing_workclass = data[data['workclass'].isin([np.nan])]
#training_workclass = data.merge(testing_workclass, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
#training_workclass.head(3)


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,salary,_merge
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,left_only
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,left_only
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,left_only


In [15]:
def create_categories_for_df(df):
    """
    df: Pandas dataframe
    returns nested dictionary {'col_name': {'val1': int, 'val2': int, 'val3': int}}
    Ex:
    {'marital.status':
     ' Married-civ-spouse': 14340,
     ' Never-married': 9916,
     ' Divorced': 4259,
     ' Separated': 959,
     ' Widowed': 840,
     ' Married-spouse-absent': 389,
     ' Married-AF-spouse': 21
     }
    """
    cols = data.columns.values.tolist()
    return_dict = {}
    for col in cols:
        return_dict[col] = dict(df[col].value_counts())
    return return_dict


In [16]:
#skip for now
my_dict = create_categories_for_df(training_workclass)

NameError: name 'training_workclass' is not defined

In [41]:
def get_categories_for_df_col(dict_of_dicts, col_name):
    """
    dict_of_dicts: dictionary containing column name for data frame and value list with frequencies
    col_name: string column name
    returns dictionary of frequencies
    """
    if col_name not in dict_of_dicts:
        print("The column", col_name, "does not exist in this nested dictionary")
        return None
    else:
        return dict_of_dicts[col_name]

In [42]:
get_categories_for_df_col(my_dict, 'marital.status')

{' Married-civ-spouse': 14340,
 ' Never-married': 9916,
 ' Divorced': 4259,
 ' Separated': 959,
 ' Widowed': 840,
 ' Married-spouse-absent': 389,
 ' Married-AF-spouse': 21}

In [17]:
# setting category for workclass 
#data['workclass'] = data['workclass'].astype('category')
#data['workclass'].dtype
#training_workclass.head()

NameError: name 'training_workclass' is not defined

In [18]:
# skip this for now 
#datawig
#imputer = datawig.SimpleImputer(
#    input_columns=[' age'], # column(s) containing information about the column we want to impute
#    output_column= 'workclass', # the column we'd like to impute values for
#    output_path = "imputer_model"
#)
#imputer = imputer.fit(train_df = training_workclass, test_df = testing_workclass)
#imputations = imputer.predict(testing_workclass)

#Impute missing values and return original dataframe with predictions
#imputed = imputer.predict(df_test)


In [ ]:
# skip
# df_train = training_workclass
# df_test = testing_workclass

# imputer = datawig.SimpleImputer(
#     input_columns=['age'], # column(s) containing information about the column we want to impute
#     output_column= 'workclass', # the column we'd like to impute values for
#     output_path = 'imputer_model' # stores model data and metrics
#     )
# imputer.fit(train_df=df_train, num_epochs=50)

# #Impute missing values and return original dataframe with predictions
# #imputed = imputer.predict(df_test)
#imputed = imputer.predict(data.loc[[26]])
#imputed.loc[[26]]

In [19]:



# use sklearn package for imputation. strategy is univariate 
#since multivariate is experimental and datawig will be a much better choice
from sklearn.impute import SimpleImputer
#imp = SimpleImputer(strategy='most_frequent')
#imp.fit(data)


fill_NaN = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputed_DF = pd.DataFrame(fill_NaN.fit_transform(data))
imputed_DF.columns = data.columns
imputed_DF.index = data.index






In [20]:
data['salary'] = data['salary'].astype('category')
data['salary'].dtype
#changing categorical data to numerical]
data[col]= data[col].replace([val],np.nan)
imputed_DF['salary'] = imputed_DF['salary'].replace([" <=50K"],0)
imputed_DF['salary'] = imputed_DF['salary'].replace([" >50K"],1)

imputed_DF

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,salary
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32556,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32557,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32558,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0


In [ ]:
### NEW INFO ###
## Correlation matrix
data.corr()

In [ ]:
var = "salary" # defining and checking descriptive stats of variable of interest
series = data[var]

In [ ]:
print("Mean: {}".format(series.mean()))
print("Stdev: {}".format(series.std()))
print("Min: {}".format(series.min()))
print("Max: {}".format(series.max()))
print("Median: {}".format(series.median()))
print("Mode: {}".format(series.mode()))
print("Skew: {}".format(series.skew()))
print("Kurtosis: {}".format(series.kurtosis()))

In [ ]:
### Graphical Summaries
## Scatterplots
sns.scatterplot(x = "age", y = "salary", data = data)

In [ ]:
sns.scatterplot(x = "sex", y = "salary", data = data)

In [ ]:
sns.scatterplot(x = "hours.per.week", y = "salary", data = data)

In [ ]:
sns.scatterplot(x = "education.num", y = "salary", data = data) # used for testing continous DV
# Note: could also use just education, too, but then have to turn strings into binary

In [ ]:
sns.pairplot(data = data)
# Q: How to zoom in on particular plots?

In [ ]:
## Boxplots
sns.boxplot(x='sex', y='salary', data= data)

In [ ]:
sns.boxplot(x='age', y='salary', data= data)

In [ ]:
sns.boxplot(x='hours.per.week', y='salary', data= data)

In [ ]:
# Get variable names

def getvar(x):
    for i in x:
        print(i)

In [ ]:
getvar(data)

In [ ]:
# Histograms
# Note: See cell 22 in Yaniv's EDA notebook

print(min(data['age']))
print(max(data['age'])) # used to check if increasing or decreasing bins is needed
data['age'].hist(bins=30)

In [ ]:
print(min(data['fnlwgt']))
print(max(data['fnlwgt'])) # used to check if increasing or decreasing bins is needed
data['fnlwgt'].hist(bins=30)

In [ ]:
print(min(data['education.num']))
print(max(data['education.num'])) # used to check if increasing or decreasing bins is needed
data['education.num'].hist(bins=30)

In [ ]:
print(min(data['capital.gain']))
print(max(data['capital.gain'])) # used to check if increasing or decreasing bins is needed
data['capital.gain'].hist(bins=30)

In [ ]:
print(min(data['capital.loss']))
print(max(data['capital.loss'])) # used to check if increasing or decreasing bins is needed
data['capital.loss'].hist(bins=30)

In [ ]:
print(min(data['hours.per.week']))
print(max(data['hours.per.week'])) # used to check if increasing or decreasing bins is needed
data['hours.per.week'].hist(bins=30)

In [ ]:
## Outliers (based on histograms... could also examine boxplots, if need be)
# Note: Can do it qualitatively or quantitatively... will be doing the former here. However, we can take the latter
# approach if we so desired: https://medium.com/datadriveninvestor/finding-outliers-in-dataset-using-python-efc3fce6ce32
# Q: Should use Judgement vs. IQR/Z-scores?

# Ex # 1: platelets variable
# Before
sns.scatterplot(x='	hours.per.week', y = 'salary', data=data)

In [ ]:
# After
# See cell 32 in Yaniv's EDA notebook

data = data[data['hours.per.week'] <= 80]
sns.scatterplot(x='hours.per.week', y = 'salary', data=data)

In [ ]:
# Ex #2: age variable
# Note: don't think we should exclude age, as study wanted to focus on all age groups, but just to see

# Before
sns.scatterplot(x='age', y = 'salary', data=data)

In [ ]:
# After

data = data[data['age'] <= 80]
sns.scatterplot(x='age', y = 'salary', data=data)

# Q: Which variables to ultimately include in predictive models? Important to know since they may need to be adjusted for outliers

In [ ]:
## Logistic Regression
## NOTE: As DEATH_EVENT is a binary variable, we must do logit; linear regression will not work
## Reference: https://gist.github.com/reenashaw/9de3608fa94f1a9ade0e19135afac70a

data['int']=1 #explicitly create a placeholder for y-intercept: b0
data.head()

In [ ]:
IV = ['age','sex','hours.per.week']
model= sm.Logit(data['salary'], data[IV])
answer= model.fit()

# Note: MUST convert sex variable into binary 1st

In [ ]:
answer.summary() #summarize the model